In [1]:
# ══════════════════════════════════════════════════════════════════
# NOTEBOOK 02: LIMPIEZA Y TRANSFORMACIÓN
# Disney Data Pipeline - Fase 2
# ══════════════════════════════════════════════════════════════════

import os
import pickle
import re
from pathlib import Path
from datetime import datetime

import pandas as pd
import numpy as np

from dotenv import load_dotenv
import boto3

import warnings
warnings.filterwarnings('ignore')

print("✅ Imports completados")
print(f"📅 {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

✅ Imports completados
📅 2025-10-17 17:04:11


In [2]:
# ══════════════════════════════════════════════════════════════════
# CELDA 2: CONFIGURACIÓN AWS
# ══════════════════════════════════════════════════════════════════

load_dotenv(override=True)

aws_session = boto3.Session(
    aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
    region_name=os.getenv('AWS_DEFAULT_REGION')
)
s3_client = aws_session.client('s3')

S3_BUCKET = os.getenv('S3_BUCKET_NAME')
S3_CLEANED_PREFIX = 'disney-project/cleaned'

def upload_to_s3(local_file, s3_key):
    """Sube archivo a S3 con encriptación"""
    try:
        s3_client.upload_file(
            Filename=local_file,
            Bucket=S3_BUCKET,
            Key=s3_key,
            ExtraArgs={'ServerSideEncryption': 'AES256'}
        )
        return f"✅ Subido: s3://{S3_BUCKET}/{s3_key}"
    except Exception as e:
        return f"❌ Error: {e}"

print("✅ AWS configurado")
print(f"   Bucket: {S3_BUCKET}")

✅ AWS configurado
   Bucket: xideralaws-curso-fernanda


In [3]:
# ══════════════════════════════════════════════════════════════════
# CELDA 3: CARGAR DATOS DE FASE 1
# ══════════════════════════════════════════════════════════════════

print("📦 Cargando datos de Fase 1...\n")
print("=" * 80)

# Verificar archivo
if not Path('datos_fase1.pkl').exists():
    raise FileNotFoundError(
        "❌ No se encontró 'datos_fase1.pkl'\n"
        "   Ejecuta primero: 01_ingesta_datos.ipynb"
    )

# Cargar pickle
with open('datos_fase1.pkl', 'rb') as f:
    datos_fase1 = pickle.load(f)

# Extraer DataFrames
df_movies = datos_fase1['df_movies'].copy()
df_characters = datos_fase1['df_characters'].copy()

print("✅ Datos cargados exitosamente")
print(f"\n📊 Datasets:")
print(f"   🎬 Películas: {len(df_movies):,} registros")
print(f"   👥 Personajes: {len(df_characters):,} registros")

# Mostrar columnas de películas
print(f"\n📋 Columnas de películas ({len(df_movies.columns)}):")
for i, col in enumerate(df_movies.columns, 1):
    print(f"   {i:2d}. {col}")

📦 Cargando datos de Fase 1...

✅ Datos cargados exitosamente

📊 Datasets:
   🎬 Películas: 119 registros
   👥 Personajes: 1,500 registros

📋 Columnas de películas (9):
    1. film_title
    2. brand
    3. box_office_revenue
    4. opening_revenue
    5. release_date
    6. opening_revenue_over_total_revenue
    7. imdb_score
    8. rt_critics_score
    9. rt_audience_score


In [4]:
# ══════════════════════════════════════════════════════════════════
# CELDA 4: LIMPIEZA DE PELÍCULAS - VALORES NULOS
# ══════════════════════════════════════════════════════════════════

print("🧹 LIMPIEZA DE PELÍCULAS\n")
print("=" * 80)

print("1️⃣ Estado inicial:")
print(f"   Registros: {len(df_movies):,}")
print(f"   Columnas: {len(df_movies.columns)}")

# Mostrar valores nulos
print(f"\n❌ Valores nulos por columna:")
null_counts = df_movies.isnull().sum()
null_counts = null_counts[null_counts > 0].sort_values(ascending=False)
if len(null_counts) > 0:
    for col, count in null_counts.items():
        pct = (count / len(df_movies)) * 100
        print(f"   {col:30} : {count:4d} ({pct:5.1f}%)")
else:
    print("   ✅ No hay valores nulos")

# Eliminar duplicados
print(f"\n2️⃣ Eliminando duplicados...")
initial_rows = len(df_movies)
df_movies = df_movies.drop_duplicates()
duplicates_removed = initial_rows - len(df_movies)
print(f"   Duplicados eliminados: {duplicates_removed}")
print(f"   Registros restantes: {len(df_movies):,}")

print("\n✅ Limpieza inicial completada")

🧹 LIMPIEZA DE PELÍCULAS

1️⃣ Estado inicial:
   Registros: 119
   Columnas: 9

❌ Valores nulos por columna:
   ✅ No hay valores nulos

2️⃣ Eliminando duplicados...
   Duplicados eliminados: 0
   Registros restantes: 119

✅ Limpieza inicial completada


In [5]:
# ══════════════════════════════════════════════════════════════════
# CELDA 5: TRANSFORMACIÓN DE COLUMNAS
# ══════════════════════════════════════════════════════════════════

print("🔧 TRANSFORMACIÓN DE COLUMNAS\n")
print("=" * 80)

# ============================================
# 1. FECHAS
# ============================================
print("1️⃣ Procesando fechas...")

# Buscar columna de fecha
date_cols = [col for col in df_movies.columns if 'date' in col.lower() or 'year' in col.lower()]
print(f"   Columnas de fecha encontradas: {date_cols}")

if date_cols:
    date_col = date_cols[0]  # Usar la primera
    
    # Convertir a datetime
    df_movies['release_date'] = pd.to_datetime(df_movies[date_col], errors='coerce')
    
    # Extraer componentes
    df_movies['release_year'] = df_movies['release_date'].dt.year
    df_movies['release_month'] = df_movies['release_date'].dt.month
    df_movies['release_quarter'] = df_movies['release_date'].dt.quarter
    df_movies['release_day_of_week'] = df_movies['release_date'].dt.day_name()
    
    print(f"   ✅ Fecha procesada desde: {date_col}")
    print(f"   ✅ Año extraído: {df_movies['release_year'].min():.0f} - {df_movies['release_year'].max():.0f}")

# ============================================
# 2. REVENUE (BOX OFFICE)
# ============================================
print("\n2️⃣ Procesando revenue...")

# Buscar columnas de revenue
revenue_cols = [col for col in df_movies.columns if 'revenue' in col.lower() or 'gross' in col.lower() or 'box' in col.lower()]
print(f"   Columnas de revenue encontradas: {revenue_cols}")

if revenue_cols:
    revenue_col = revenue_cols[0]
    
    # Función para limpiar valores monetarios
    def clean_revenue(value):
        if pd.isna(value):
            return np.nan
        # Convertir a string y limpiar
        value_str = str(value).replace('$', '').replace(',', '').replace(' ', '')
        try:
            return float(value_str)
        except:
            return np.nan
    
    df_movies['box_office_revenue_clean'] = df_movies[revenue_col].apply(clean_revenue)
    
    print(f"   ✅ Revenue limpiado desde: {revenue_col}")
    print(f"   ✅ Rango: ${df_movies['box_office_revenue_clean'].min():,.0f} - ${df_movies['box_office_revenue_clean'].max():,.0f}")

# ============================================
# 3. RATINGS (IMDB, RT)
# ============================================
print("\n3️⃣ Procesando ratings...")

# Buscar columnas de rating
rating_cols = [col for col in df_movies.columns if 'rating' in col.lower() or 'score' in col.lower() or 'imdb' in col.lower()]
print(f"   Columnas de rating encontradas: {rating_cols}")

for col in rating_cols:
    if df_movies[col].dtype == 'object':
        # Limpiar y convertir a numérico
        df_movies[col] = pd.to_numeric(df_movies[col].astype(str).str.replace('%', ''), errors='coerce')
        print(f"   ✅ {col} convertido a numérico")

print("\n✅ Transformaciones completadas")

🔧 TRANSFORMACIÓN DE COLUMNAS

1️⃣ Procesando fechas...
   Columnas de fecha encontradas: ['release_date']
   ✅ Fecha procesada desde: release_date
   ✅ Año extraído: 1989 - 2023

2️⃣ Procesando revenue...
   Columnas de revenue encontradas: ['box_office_revenue', 'opening_revenue', 'opening_revenue_over_total_revenue']
   ✅ Revenue limpiado desde: box_office_revenue
   ✅ Rango: $19,478,106 - $936,662,225

3️⃣ Procesando ratings...
   Columnas de rating encontradas: ['imdb_score', 'rt_critics_score', 'rt_audience_score']

✅ Transformaciones completadas


In [6]:
# ══════════════════════════════════════════════════════════════════
# CELDA 6: CREAR COLUMNAS CALCULADAS
# ══════════════════════════════════════════════════════════════════

print("➕ CREANDO COLUMNAS CALCULADAS\n")
print("=" * 80)

# ============================================
# 1. DÉCADAS
# ============================================
print("1️⃣ Creando columna de década...")
if 'release_year' in df_movies.columns:
    df_movies['decade'] = (df_movies['release_year'] // 10) * 10
    df_movies['decade_label'] = df_movies['decade'].astype(str) + 's'
    
    print(f"   ✅ Décadas creadas:")
    print(df_movies['decade_label'].value_counts().sort_index())

# ============================================
# 2. CATEGORÍAS DE RATING
# ============================================
print("\n2️⃣ Creando categorías de rating...")

# Buscar columna IMDB
imdb_col = [col for col in df_movies.columns if 'imdb' in col.lower()]
if imdb_col:
    imdb_col = imdb_col[0]
    
    df_movies['rating_category'] = pd.cut(
        df_movies[imdb_col],
        bins=[0, 5, 6.5, 7.5, 10],
        labels=['Bajo', 'Medio', 'Alto', 'Excelente']
    )
    
    print(f"   ✅ Categorías de rating creadas:")
    print(df_movies['rating_category'].value_counts())

# ============================================
# 3. SEGMENTACIÓN DE PELÍCULAS
# ============================================
print("\n3️⃣ Creando segmentación de películas...")

def segment_movie(row):
    """Segmenta películas por rating y revenue"""
    # Buscar columnas
    imdb_col = [col for col in row.index if 'imdb' in col.lower()]
    revenue_col = 'box_office_revenue_clean'
    
    if not imdb_col or revenue_col not in row.index:
        return 'Sin Clasificar'
    
    imdb_col = imdb_col[0]
    
    rating = row[imdb_col]
    revenue = row[revenue_col]
    
    if pd.isna(rating) or pd.isna(revenue):
        return 'Sin Clasificar'
    
    high_rating = rating >= 7.0
    high_revenue = revenue >= 300_000_000
    
    if high_rating and high_revenue:
        return 'Éxito Crítico y Comercial'
    elif high_rating:
        return 'Éxito Crítico'
    elif high_revenue:
        return 'Éxito Comercial'
    else:
        return 'Bajo Rendimiento'

df_movies['segment'] = df_movies.apply(segment_movie, axis=1)

print(f"   ✅ Segmentación creada:")
print(df_movies['segment'].value_counts())

print("\n✅ Columnas calculadas creadas")

➕ CREANDO COLUMNAS CALCULADAS

1️⃣ Creando columna de década...
   ✅ Décadas creadas:
decade_label
1980s     1
1990s    14
2000s    21
2010s    63
2020s    20
Name: count, dtype: int64

2️⃣ Creando categorías de rating...
   ✅ Categorías de rating creadas:
rating_category
Alto         63
Excelente    35
Medio        20
Bajo          1
Name: count, dtype: int64

3️⃣ Creando segmentación de películas...
   ✅ Segmentación creada:
segment
Éxito Crítico                43
Bajo Rendimiento             38
Éxito Crítico y Comercial    30
Éxito Comercial               8
Name: count, dtype: int64

✅ Columnas calculadas creadas


In [7]:
# ══════════════════════════════════════════════════════════════════
# CELDA 7: NORMALIZAR NOMBRES DE PELÍCULAS
# ══════════════════════════════════════════════════════════════════

print("🔤 NORMALIZANDO NOMBRES\n")
print("=" * 80)

def normalize_title(title):
    """Normaliza título de película para matching"""
    if pd.isna(title):
        return ''
    # Convertir a minúsculas
    title = str(title).lower()
    # Remover caracteres especiales
    title = re.sub(r'[^\w\s]', '', title)
    # Remover espacios múltiples
    title = re.sub(r'\s+', ' ', title)
    # Trim
    title = title.strip()
    return title

# Buscar columna de título
title_cols = [col for col in df_movies.columns if 'title' in col.lower() or 'movie' in col.lower() or 'film' in col.lower()]
if title_cols:
    title_col = title_cols[0]
    df_movies['film_title'] = df_movies[title_col]
    df_movies['film_title_clean'] = df_movies[title_col].apply(normalize_title)
    
    print(f"✅ Títulos normalizados desde: {title_col}")
    print(f"\n📋 Ejemplos:")
    print(df_movies[['film_title', 'film_title_clean']].head(5).to_string(index=False))
else:
    print("⚠️  No se encontró columna de título")

print("\n✅ Normalización completada")

🔤 NORMALIZANDO NOMBRES

✅ Títulos normalizados desde: film_title

📋 Ejemplos:
                                 film_title                            film_title_clean
                                     Onward                                      onward
                                The Marvels                                 the marvels
          Ant-Man and the Wasp: Quantumania             antman and the wasp quantumania
Doctor Strange in the Multiverse of Madness doctor strange in the multiverse of madness
                 Captain America: Civil War                   captain america civil war

✅ Normalización completada


In [8]:
# ══════════════════════════════════════════════════════════════════
# CELDA 8: LIMPIEZA DE PERSONAJES
# ══════════════════════════════════════════════════════════════════

print("🧹 LIMPIEZA DE PERSONAJES\n")
print("=" * 80)

print("1️⃣ Estado inicial:")
print(f"   Registros: {len(df_characters):,}")
print(f"   Columnas: {len(df_characters.columns)}")

# Eliminar duplicados
initial_chars = len(df_characters)
df_characters = df_characters.drop_duplicates(subset=['name'] if 'name' in df_characters.columns else None)
print(f"\n2️⃣ Duplicados eliminados: {initial_chars - len(df_characters)}")

# Contar apariciones
if 'films' in df_characters.columns:
    df_characters['num_films'] = df_characters['films'].apply(
        lambda x: len(x) if isinstance(x, list) else 0
    )
    print(f"\n3️⃣ Películas por personaje:")
    print(f"   Promedio: {df_characters['num_films'].mean():.1f}")
    print(f"   Máximo: {df_characters['num_films'].max()}")
    print(f"   Sin películas: {(df_characters['num_films'] == 0).sum()}")

if 'tvShows' in df_characters.columns:
    df_characters['num_tv_shows'] = df_characters['tvShows'].apply(
        lambda x: len(x) if isinstance(x, list) else 0
    )

# Total de apariciones
if 'num_films' in df_characters.columns and 'num_tv_shows' in df_characters.columns:
    df_characters['total_appearances'] = df_characters['num_films'] + df_characters['num_tv_shows']
    
    # Categoría de popularidad
    df_characters['popularity_category'] = pd.cut(
        df_characters['total_appearances'],
        bins=[-1, 0, 5, 15, 100],
        labels=['Sin Apariciones', 'Baja', 'Media', 'Alta']
    )
    
    print(f"\n4️⃣ Categorías de popularidad:")
    print(df_characters['popularity_category'].value_counts())

print("\n✅ Limpieza de personajes completada")

🧹 LIMPIEZA DE PERSONAJES

1️⃣ Estado inicial:
   Registros: 1,500
   Columnas: 13

2️⃣ Duplicados eliminados: 81

3️⃣ Películas por personaje:
   Promedio: 0.8
   Máximo: 10
   Sin películas: 771

4️⃣ Categorías de popularidad:
popularity_category
Baja               1248
Sin Apariciones     113
Media                54
Alta                  4
Name: count, dtype: int64

✅ Limpieza de personajes completada


In [10]:
# ══════════════════════════════════════════════════════════════════
# CELDA 9: CREAR RELACIONES PELÍCULA-PERSONAJE
# ══════════════════════════════════════════════════════════════════

print("🔗 CREANDO RELACIONES PELÍCULA-PERSONAJE\n")
print("=" * 80)

relations = []

if 'films' in df_characters.columns and 'name' in df_characters.columns:
    print("📊 Procesando relaciones...")
    
    for idx, row in df_characters.iterrows():
        character_name = row['name']
        films = row['films']
        
        if isinstance(films, list):
            for film in films:
                relations.append({
                    'character_name': character_name,
                    'movie_title': film,
                    'movie_title_clean': normalize_title(film)
                })
    
    df_relations = pd.DataFrame(relations)
    
    print(f"\n✅ Relaciones creadas:")
    print(f"   Total relaciones: {len(df_relations):,}")
    print(f"   Personajes únicos: {df_relations['character_name'].nunique():,}")
    print(f"   Películas únicas: {df_relations['movie_title'].nunique():,}")
    
    # Top películas con más personajes
    print(f"\n🎬 Top 10 películas con más personajes:")
    top_movies = df_relations['movie_title'].value_counts().head(10)
    for movie, count in top_movies.items():
        print(f"   {movie[:40]:40} : {count:3d} personajes")
    
else:
    print("⚠️  No se pueden crear relaciones (faltan columnas)")
    df_relations = pd.DataFrame()

print("\n✅ Relaciones completadas")

🔗 CREANDO RELACIONES PELÍCULA-PERSONAJE

📊 Procesando relaciones...

✅ Relaciones creadas:
   Total relaciones: 1,068
   Personajes únicos: 648
   Películas únicas: 368

🎬 Top 10 películas con más personajes:
   Who Framed Roger Rabbit                  :  27 personajes
   Mickey's Magical Christmas: Snowed in at :  26 personajes
   Mickey's House of Villains               :  20 personajes
   Ralph Breaks the Internet                :  19 personajes
   Leroy & Stitch                           :  18 personajes
   Once Upon a Halloween                    :  13 personajes
   Pirates of the Caribbean: At World's End :  13 personajes
   Fantasia                                 :  12 personajes
   Pirates of the Caribbean: Dead Man's Che :  11 personajes
   Alice in Wonderland (2010 film)          :  10 personajes

✅ Relaciones completadas


In [11]:
# ══════════════════════════════════════════════════════════════════
# CELDA 10: GUARDAR DATOS LIMPIOS LOCALMENTE
# ══════════════════════════════════════════════════════════════════

print("💾 GUARDANDO DATOS LIMPIOS LOCALMENTE\n")
print("=" * 80)

# Crear directorio
Path('data/cleaned').mkdir(parents=True, exist_ok=True)

# 1. Películas
movies_path = 'data/cleaned/movies_cleaned.csv'
df_movies.to_csv(movies_path, index=False, encoding='utf-8')
print(f"✅ {movies_path}")
print(f"   Registros: {len(df_movies):,}")
print(f"   Columnas: {len(df_movies.columns)}")

# 2. Personajes
characters_path = 'data/cleaned/characters_cleaned.csv'
df_characters.to_csv(characters_path, index=False, encoding='utf-8')
print(f"\n✅ {characters_path}")
print(f"   Registros: {len(df_characters):,}")
print(f"   Columnas: {len(df_characters.columns)}")

# 3. Relaciones
if not df_relations.empty:
    relations_path = 'data/cleaned/relations.csv'
    df_relations.to_csv(relations_path, index=False, encoding='utf-8')
    print(f"\n✅ {relations_path}")
    print(f"   Registros: {len(df_relations):,}")
    print(f"   Columnas: {len(df_relations.columns)}")

print("\n✅ Archivos guardados localmente")

💾 GUARDANDO DATOS LIMPIOS LOCALMENTE

✅ data/cleaned/movies_cleaned.csv
   Registros: 119
   Columnas: 19

✅ data/cleaned/characters_cleaned.csv
   Registros: 1,419
   Columnas: 16

✅ data/cleaned/relations.csv
   Registros: 1,068
   Columnas: 3

✅ Archivos guardados localmente


In [12]:
# ══════════════════════════════════════════════════════════════════
# CELDA 11: SUBIR DATOS LIMPIOS A S3
# ══════════════════════════════════════════════════════════════════

print("☁️  SUBIENDO DATOS LIMPIOS A S3\n")
print("=" * 80)

# 1. Películas
s3_key = f"{S3_CLEANED_PREFIX}/movies_cleaned.csv"
result = upload_to_s3(movies_path, s3_key)
print(result)

# 2. Personajes
s3_key = f"{S3_CLEANED_PREFIX}/characters_cleaned.csv"
result = upload_to_s3(characters_path, s3_key)
print(result)

# 3. Relaciones
if not df_relations.empty:
    s3_key = f"{S3_CLEANED_PREFIX}/relations.csv"
    result = upload_to_s3(relations_path, s3_key)
    print(result)

print("\n✅ Archivos subidos a S3")

☁️  SUBIENDO DATOS LIMPIOS A S3

✅ Subido: s3://xideralaws-curso-fernanda/disney-project/cleaned/movies_cleaned.csv
✅ Subido: s3://xideralaws-curso-fernanda/disney-project/cleaned/characters_cleaned.csv
✅ Subido: s3://xideralaws-curso-fernanda/disney-project/cleaned/relations.csv

✅ Archivos subidos a S3


In [13]:
# ══════════════════════════════════════════════════════════════════
# CELDA 12: ANÁLISIS EXPLORATORIO POST-LIMPIEZA
# ══════════════════════════════════════════════════════════════════

print("📊 ANÁLISIS EXPLORATORIO POST-LIMPIEZA\n")
print("=" * 80)

# ═══════════════════════════════════════
# PELÍCULAS POR DÉCADA
# ═══════════════════════════════════════
if 'decade_label' in df_movies.columns:
    print("\n🎬 PELÍCULAS POR DÉCADA:")
    print("-" * 80)
    decade_counts = df_movies['decade_label'].value_counts().sort_index()
    for decade, count in decade_counts.items():
        print(f"   {decade:10} : {count:3d} películas")

# ═══════════════════════════════════════
# PELÍCULAS POR SEGMENTO
# ═══════════════════════════════════════
if 'segment' in df_movies.columns:
    print("\n📈 PELÍCULAS POR SEGMENTO:")
    print("-" * 80)
    segment_counts = df_movies['segment'].value_counts()
    for segment, count in segment_counts.items():
        pct = (count / len(df_movies)) * 100
        print(f"   {segment:30} : {count:3d} ({pct:5.1f}%)")

# ═══════════════════════════════════════
# TOP PELÍCULAS POR REVENUE
# ═══════════════════════════════════════
if 'box_office_revenue_clean' in df_movies.columns and 'film_title' in df_movies.columns:
    print("\n💰 TOP 10 PELÍCULAS POR REVENUE:")
    print("-" * 80)
    top_revenue = df_movies.nlargest(10, 'box_office_revenue_clean')[['film_title', 'box_office_revenue_clean', 'release_year']]
    for idx, row in top_revenue.iterrows():
        print(f"   {row['film_title'][:35]:35} : ${row['box_office_revenue_clean']:>13,.0f} ({row['release_year']:.0f})")

# ═══════════════════════════════════════
# ESTADÍSTICAS GENERALES
# ═══════════════════════════════════════
print("\n📊 ESTADÍSTICAS GENERALES:")
print("-" * 80)

if 'box_office_revenue_clean' in df_movies.columns:
    print(f"   Revenue promedio: ${df_movies['box_office_revenue_clean'].mean():,.0f}")
    print(f"   Revenue total: ${df_movies['box_office_revenue_clean'].sum():,.0f}")

imdb_col = [col for col in df_movies.columns if 'imdb' in col.lower()]
if imdb_col:
    print(f"   Rating IMDB promedio: {df_movies[imdb_col[0]].mean():.2f}")

if 'release_year' in df_movies.columns:
    print(f"   Año más antiguo: {df_movies['release_year'].min():.0f}")
    print(f"   Año más reciente: {df_movies['release_year'].max():.0f}")

print("\n✅ Análisis completado")

📊 ANÁLISIS EXPLORATORIO POST-LIMPIEZA


🎬 PELÍCULAS POR DÉCADA:
--------------------------------------------------------------------------------
   1980s      :   1 películas
   1990s      :  14 películas
   2000s      :  21 películas
   2010s      :  63 películas
   2020s      :  20 películas

📈 PELÍCULAS POR SEGMENTO:
--------------------------------------------------------------------------------
   Éxito Crítico                  :  43 ( 36.1%)
   Bajo Rendimiento               :  38 ( 31.9%)
   Éxito Crítico y Comercial      :  30 ( 25.2%)
   Éxito Comercial                :   8 (  6.7%)

💰 TOP 10 PELÍCULAS POR REVENUE:
--------------------------------------------------------------------------------
   Star Wars: Episode VII - The Force  : $  936,662,225 (2015)
   Avengers: Endgame                   : $  858,373,000 (2019)
   Spider-Man: No Way Home             : $  804,793,477 (2021)
   Black Panther                       : $  700,059,566 (2018)
   Avengers: Infinity War          

In [14]:
# ══════════════════════════════════════════════════════════════════
# CELDA 13: GUARDAR DATOS PARA FASE 3 (SPARK)
# ══════════════════════════════════════════════════════════════════

print("📦 GUARDANDO DATOS PARA FASE 3\n")
print("=" * 80)

# Empaquetar datos limpios
datos_fase2 = {
    'df_movies_clean': df_movies,
    'df_characters_clean': df_characters,
    'df_relations': df_relations,
    'metadata': {
        'movies_count': len(df_movies),
        'characters_count': len(df_characters),
        'relations_count': len(df_relations),
        'movies_columns': list(df_movies.columns),
        'characters_columns': list(df_characters.columns),
        'cleaning_date': datetime.now().isoformat(),
        'notebook': '02_limpieza_transformacion.ipynb',
        'status': 'SUCCESS'
    }
}

# Guardar pickle
with open('datos_fase2.pkl', 'wb') as f:
    pickle.dump(datos_fase2, f)

print("✅ Pickle guardado: datos_fase2.pkl")
print(f"\n📊 Contenido:")
print(f"   🎬 df_movies_clean: {len(df_movies):,} registros, {len(df_movies.columns)} columnas")
print(f"   👥 df_characters_clean: {len(df_characters):,} registros, {len(df_characters.columns)} columnas")
print(f"   🔗 df_relations: {len(df_relations):,} registros, {len(df_relations.columns)} columnas")

📦 GUARDANDO DATOS PARA FASE 3

✅ Pickle guardado: datos_fase2.pkl

📊 Contenido:
   🎬 df_movies_clean: 119 registros, 19 columnas
   👥 df_characters_clean: 1,419 registros, 16 columnas
   🔗 df_relations: 1,068 registros, 3 columnas


In [15]:
# ══════════════════════════════════════════════════════════════════
# CELDA 14: RESUMEN FINAL DEL NOTEBOOK
# ══════════════════════════════════════════════════════════════════

print("\n" + "=" * 80)
print("🎉 NOTEBOOK 02: LIMPIEZA Y TRANSFORMACIÓN - COMPLETADO")
print("=" * 80)

print(f"\n📊 TRANSFORMACIONES APLICADAS:")
print(f"   ✅ Limpieza de valores nulos")
print(f"   ✅ Eliminación de duplicados")
print(f"   ✅ Conversión de fechas")
print(f"   ✅ Limpieza de revenue")
print(f"   ✅ Normalización de ratings")
print(f"   ✅ Creación de décadas")
print(f"   ✅ Categorización de películas")
print(f"   ✅ Segmentación por performance")
print(f"   ✅ Relaciones película-personaje")

print(f"\n💾 ARCHIVOS GENERADOS:")
print(f"   ✅ data/cleaned/movies_cleaned.csv")
print(f"   ✅ data/cleaned/characters_cleaned.csv")
print(f"   ✅ data/cleaned/relations.csv")
print(f"   ✅ datos_fase2.pkl")

print(f"\n☁️  ARCHIVOS EN S3:")
print(f"   ✅ {S3_CLEANED_PREFIX}/movies_cleaned.csv")
print(f"   ✅ {S3_CLEANED_PREFIX}/characters_cleaned.csv")
print(f"   ✅ {S3_CLEANED_PREFIX}/relations.csv")

print(f"\n📊 DATOS FINALES:")
print(f"   🎬 Películas: {len(df_movies):,} registros")
print(f"   👥 Personajes: {len(df_characters):,} registros")
print(f"   🔗 Relaciones: {len(df_relations):,} registros")

print(f"\n🚀 SIGUIENTE PASO:")
print(f"   Ejecutar: 03b_procesamiento_spark.ipynb")

print("=" * 80)


🎉 NOTEBOOK 02: LIMPIEZA Y TRANSFORMACIÓN - COMPLETADO

📊 TRANSFORMACIONES APLICADAS:
   ✅ Limpieza de valores nulos
   ✅ Eliminación de duplicados
   ✅ Conversión de fechas
   ✅ Limpieza de revenue
   ✅ Normalización de ratings
   ✅ Creación de décadas
   ✅ Categorización de películas
   ✅ Segmentación por performance
   ✅ Relaciones película-personaje

💾 ARCHIVOS GENERADOS:
   ✅ data/cleaned/movies_cleaned.csv
   ✅ data/cleaned/characters_cleaned.csv
   ✅ data/cleaned/relations.csv
   ✅ datos_fase2.pkl

☁️  ARCHIVOS EN S3:
   ✅ disney-project/cleaned/movies_cleaned.csv
   ✅ disney-project/cleaned/characters_cleaned.csv
   ✅ disney-project/cleaned/relations.csv

📊 DATOS FINALES:
   🎬 Películas: 119 registros
   👥 Personajes: 1,419 registros
   🔗 Relaciones: 1,068 registros

🚀 SIGUIENTE PASO:
   Ejecutar: 03b_procesamiento_spark.ipynb
